In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import time
from collections import deque

# ========================
# UCI-HAR 데이터 로드
# ========================
class UCIHARDataset(Dataset):
    def __init__(self, data_path, split='train'):
        base = Path(data_path) / split
        signals = []
        for sensor in ['body_acc', 'body_gyro', 'total_acc']:
            for axis in ['x', 'y', 'z']:
                file = base / 'Inertial Signals' / f'{sensor}_{axis}_{split}.txt'
                signals.append(np.loadtxt(file))

        self.X = np.stack(signals, axis=-1)
        self.y = np.loadtxt(base.parent / split / f'y_{split}.txt').astype(int) - 1

        try:
            self.subjects = np.loadtxt(base.parent / split / f'subject_{split}.txt').astype(int)
        except:
            self.subjects = np.ones(len(self.y))

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (torch.FloatTensor(self.X[idx]),
                torch.LongTensor([self.y[idx]])[0],
                self.subjects[idx])

# ========================
# 🔥 Modern TCN Components
# ========================
class DepthwiseSeparableConv1d(nn.Module):
    """Depthwise Separable Convolution for efficiency"""
    def __init__(self, in_channels, out_channels, kernel_size, dilation=1, padding=0):
        super().__init__()
        self.depthwise = nn.Conv1d(
            in_channels, in_channels, kernel_size,
            padding=padding, dilation=dilation, groups=in_channels
        )
        self.pointwise = nn.Conv1d(in_channels, out_channels, 1)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class MultiScaleConvBlock(nn.Module):
    """Multi-scale convolution block with small and large kernels"""
    def __init__(self, channels, kernel_sizes=[3, 5, 7], dilation=1, dropout=0.1):
        super().__init__()

        self.branches = nn.ModuleList()
        for k in kernel_sizes:
            # Use 'same' padding to maintain length
            padding = ((k - 1) * dilation) // 2
            branch = nn.ModuleDict({
                'conv': DepthwiseSeparableConv1d(channels, channels, k, dilation, padding),
                'norm': nn.BatchNorm1d(channels),  # Use BatchNorm1d
                'dropout': nn.Dropout(dropout)
            })
            self.branches.append(branch)

        # Fusion layer
        self.fusion = nn.Conv1d(channels * len(kernel_sizes), channels, 1)

    def forward(self, x):
        # x: [B, C, L]
        outputs = []
        target_length = x.size(2)  # Store original length

        for branch in self.branches:
            out = branch['conv'](x)
            # Ensure all outputs have the same length
            if out.size(2) != target_length:
                out = out[:, :, :target_length]
            out = branch['norm'](out)  # BatchNorm1d works with [B, C, L]
            out = F.gelu(out)
            out = branch['dropout'](out)
            outputs.append(out)

        # Concatenate and fuse
        multi_scale = torch.cat(outputs, dim=1)
        return self.fusion(multi_scale)

class ModernTCNBlock(nn.Module):
    """Modern TCN Block with:
    - Multi-scale convolutions (small and large kernels)
    - Batch Normalization (for Conv1d compatibility)
    - GELU activation
    - Residual connections
    """
    def __init__(self, in_channels, out_channels, kernel_sizes=[3, 7], dilation=1, dropout=0.1):
        super().__init__()

        # First multi-scale conv
        self.multi_conv1 = MultiScaleConvBlock(
            in_channels if in_channels == out_channels else out_channels,
            kernel_sizes, dilation, dropout
        )

        # Standard depthwise separable conv
        max_k = max(kernel_sizes)
        padding = ((max_k - 1) * dilation) // 2
        self.conv2 = DepthwiseSeparableConv1d(
            out_channels, out_channels, max_k, dilation, padding
        )
        self.norm2 = nn.BatchNorm1d(out_channels)  # Use BatchNorm1d
        self.dropout2 = nn.Dropout(dropout)

        # Residual connection
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None

    def forward(self, x):
        # x: [B, C, L]
        residual = x
        target_length = x.size(2)

        # Adjust channels if needed
        if self.downsample is not None:
            x = self.downsample(x)
            residual = x

        # Multi-scale convolution
        out = self.multi_conv1(x)
        if out.size(2) != target_length:
            out = out[:, :, :target_length]

        # Standard convolution
        out = self.conv2(out)
        if out.size(2) != target_length:
            out = out[:, :, :target_length]
        out = self.norm2(out)  # BatchNorm1d works with [B, C, L]
        out = F.gelu(out)
        out = self.dropout2(out)

        # Residual connection
        return F.gelu(out + residual)

class SqueezeExcitation1d(nn.Module):
    """Squeeze-and-Excitation block for channel attention"""
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.fc2 = nn.Linear(channels // reduction, channels)

    def forward(self, x):
        # x: [B, C, L]
        batch, channels, _ = x.size()

        # Squeeze: Global Average Pooling
        squeeze = F.adaptive_avg_pool1d(x, 1).view(batch, channels)

        # Excitation
        excitation = F.relu(self.fc1(squeeze))
        excitation = torch.sigmoid(self.fc2(excitation)).view(batch, channels, 1)

        return x * excitation

class LargeKernelConv1d(nn.Module):
    """Large kernel convolution decomposed into depthwise and pointwise"""
    def __init__(self, channels, kernel_size=21):
        super().__init__()
        padding = kernel_size // 2
        self.depthwise = nn.Conv1d(
            channels, channels, kernel_size,
            padding=padding, groups=channels
        )
        self.norm = nn.BatchNorm1d(channels)  # Use BatchNorm1d instead of LayerNorm

    def forward(self, x):
        # x: [B, C, L]
        out = self.depthwise(x)
        out = self.norm(out)  # BatchNorm1d works with [B, C, L]
        return out

# ========================
# Modern TCN Base 모델 (Multi-scale)
# ========================
class BaseModernTCNHAR(nn.Module):
    def __init__(self, input_dim=9, hidden_dim=128, n_layers=4, n_classes=6,
                 kernel_sizes=[3, 7], large_kernel=21, dropout=0.1, use_se=True):
        super().__init__()

        # Input projection
        self.input_proj = nn.Conv1d(input_dim, hidden_dim, 1)

        # Large kernel at the beginning for global context
        self.large_kernel_conv = LargeKernelConv1d(hidden_dim, large_kernel)

        # TCN blocks with exponentially increasing dilation and multi-scale kernels
        self.tcn_blocks = nn.ModuleList()
        for i in range(n_layers):
            dilation = 2 ** i
            self.tcn_blocks.append(
                ModernTCNBlock(
                    hidden_dim, hidden_dim,
                    kernel_sizes=kernel_sizes,
                    dilation=dilation,
                    dropout=dropout
                )
            )

        # Additional large kernel at the end for global aggregation
        self.final_large_kernel = LargeKernelConv1d(hidden_dim, large_kernel)

        # Optional Squeeze-and-Excitation
        self.use_se = use_se
        if use_se:
            self.se = SqueezeExcitation1d(hidden_dim)

        # Global pooling and classification head
        self.norm_final = nn.LayerNorm(hidden_dim)
        self.head = nn.Linear(hidden_dim, n_classes)

    def forward(self, x):
        # x: [B, L, C] -> [B, C, L] for Conv1d
        x = x.transpose(1, 2)

        # Input projection
        x = self.input_proj(x)

        # Initial large kernel for global context
        x = self.large_kernel_conv(x)
        x = F.gelu(x)

        # TCN blocks with multi-scale kernels
        for block in self.tcn_blocks:
            x = block(x)

        # Final large kernel for global aggregation
        x = self.final_large_kernel(x)
        x = F.gelu(x)

        # Squeeze-and-Excitation
        if self.use_se:
            x = self.se(x)

        # Global average pooling
        x = F.adaptive_avg_pool1d(x, 1).squeeze(-1)  # [B, C]

        # Layer norm and classification
        x = self.norm_final(x)
        return self.head(x)

# ========================
# Physics-Guided Modern TCN HAR
# ========================
class PhysicsModernTCNHAR(BaseModernTCNHAR):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        hidden_dim = self.head.in_features
        # 보조 태스크를 위한 '물리 헤드' 추가
        self.physics_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 6)  # 6 = acc (x,y,z) + gyro (x,y,z)
        )

    def forward(self, x, return_physics=False):
        # x: [B, L, C] -> [B, C, L] for Conv1d
        x = x.transpose(1, 2)

        # Input projection
        x_feat = self.input_proj(x)

        # Initial large kernel
        x_feat = self.large_kernel_conv(x_feat)
        x_feat = F.gelu(x_feat)

        # TCN blocks
        for block in self.tcn_blocks:
            x_feat = block(x_feat)

        # Final large kernel
        x_feat = self.final_large_kernel(x_feat)
        x_feat = F.gelu(x_feat)

        # Squeeze-and-Excitation
        if self.use_se:
            x_feat = self.se(x_feat)

        # x_feat: [B, C, L]

        # 1. 분류 헤드 (기본 태스크)
        pooled = F.adaptive_avg_pool1d(x_feat, 1).squeeze(-1)  # [B, C]
        pooled = self.norm_final(pooled)
        logits = self.head(pooled)

        if return_physics:
            # 2. 물리 헤드 (보조 태스크)
            # [B, C, L] -> [B, L, C] -> [B, L, 6]
            x_feat_transposed = x_feat.transpose(1, 2)
            physics = self.physics_head(x_feat_transposed)
            return logits, physics

        return logits

# ========================
# '물리 손실' 함수
# ========================
def physics_loss(physics_pred, X_raw):
    # physics_pred: [B, L, 6]
    # X_raw: [B, L, 9] (원본 입력)

    acc_pred = physics_pred[:, :, :3]
    gyro_pred = physics_pred[:, :, 3:6]

    acc_true = X_raw[:, :, 0:3]
    gyro_true = X_raw[:, :, 3:6]

    return F.smooth_l1_loss(acc_pred, acc_true) + F.smooth_l1_loss(gyro_pred, gyro_true)

# ========================
# '물리 기반' 학습 함수
# ========================
def train_physics(model, train_loader, test_loader, device, epochs=50):
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

    best_acc = 0

    print("="*50)
    print("🔥 Physics-Guided (Multi-scale Modern TCN) HAR 학습 시작")
    print("="*50)

    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        for X, y, _ in train_loader:
            X, y = X.to(device), y.to(device)

            optimizer.zero_grad()

            logits, physics = model(X, return_physics=True)

            loss_cls = F.cross_entropy(logits, y)
            loss_phys = physics_loss(physics, X)
            loss = loss_cls + 0.05 * loss_phys

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()
            correct += (logits.argmax(1) == y).sum().item()
            total += y.size(0)

        scheduler.step()

        model.eval()
        test_correct, test_total = 0, 0

        with torch.no_grad():
            for X, y, _ in test_loader:
                X, y = X.to(device), y.to(device)
                logits = model(X, return_physics=False)
                test_correct += (logits.argmax(1) == y).sum().item()
                test_total += y.size(0)

        train_acc = 100 * correct / total
        test_acc = 100 * test_correct / test_total
        best_acc = max(best_acc, test_acc)

        if (epoch + 1) % 10 == 0 or epoch == epochs - 1:
            print(f'[Physics] Epoch {epoch+1:02d}/{epochs}: Train Acc={train_acc:.2f}%, Test Acc={test_acc:.2f}% (Best: {best_acc:.2f}%)')

    return best_acc

# ========================
# 메인 실행 함수
# ========================
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # !!! 경로 수정 필요 !!!
    data_path = '/content/drive/MyDrive/Colab Notebooks/UCI-HAR/UCI-HAR'
    # data_path = './UCI-HAR' # 로컬 환경 예시

    try:
        train_ds = UCIHARDataset(data_path, split='train')
        test_ds = UCIHARDataset(data_path, split='test')
    except FileNotFoundError:
        print(f"Error: 데이터셋을 찾을 수 없습니다. 'data_path' 변수를 수정하세요.")
        print(f"현재 경로: {data_path}")
        return

    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

    print(f"Train: {len(train_ds)} samples, Test: {len(test_ds)} samples\n")

    # 🔥 Physics-Guided Multi-scale Modern TCN 모델 생성 및 학습
    model_physics = PhysicsModernTCNHAR(
        input_dim=9,            # UCI-HAR 9 채널
        hidden_dim=128,         # 은닉층 차원
        n_layers=4,             # TCN 블록 개수
        n_classes=6,            # UCI-HAR 6개 클래스
        kernel_sizes=[3, 7],    # Multi-scale: 작은 커널(3) + 큰 커널(7)
        large_kernel=21,        # 매우 큰 커널 (글로벌 컨텍스트용)
        dropout=0.1,            # Dropout 비율
        use_se=True             # Squeeze-and-Excitation 사용
    ).to(device)

    print(f"\n📊 모델 구조:")
    print(f"  - Multi-scale kernels: [3, 7] (작은 커널 + 큰 커널)")
    print(f"  - Large kernel: 21 (글로벌 컨텍스트)")
    print(f"  - Hidden dim: 128")
    print(f"  - Layers: 4")
    print(f"  - Squeeze-Excitation: Enabled")
    print(f"  - Total parameters: {sum(p.numel() for p in model_physics.parameters()):,}\n")

    acc_physics = train_physics(model_physics, train_loader, test_loader, device, epochs=50)

    # Summary
    print("\n" + "="*50)
    print("FINAL RESULT")
    print("="*50)
    print(f"🔥 Physics-Guided Multi-scale Modern TCN HAR (Best Test Acc): {acc_physics:.2f}%")
    print(f"   - Small kernels (3, 7) for local patterns")
    print(f"   - Large kernel (21) for global context")
    print(f"   - Physics-guided learning with auxiliary task")

if __name__ == '__main__':
    main()

Using device: cuda
Train: 7352 samples, Test: 2947 samples


📊 모델 구조:
  - Multi-scale kernels: [3, 7] (작은 커널 + 큰 커널)
  - Large kernel: 21 (글로벌 컨텍스트)
  - Hidden dim: 128
  - Layers: 4
  - Squeeze-Excitation: Enabled
  - Total parameters: 362,324

🔥 Physics-Guided (Multi-scale Modern TCN) HAR 학습 시작
[Physics] Epoch 10/50: Train Acc=96.67%, Test Acc=95.66% (Best: 95.69%)
[Physics] Epoch 20/50: Train Acc=98.67%, Test Acc=97.39% (Best: 97.39%)
[Physics] Epoch 30/50: Train Acc=99.59%, Test Acc=97.35% (Best: 97.69%)
[Physics] Epoch 40/50: Train Acc=99.76%, Test Acc=97.01% (Best: 97.69%)
[Physics] Epoch 50/50: Train Acc=99.82%, Test Acc=96.81% (Best: 97.69%)

FINAL RESULT
🔥 Physics-Guided Multi-scale Modern TCN HAR (Best Test Acc): 97.69%
   - Small kernels (3, 7) for local patterns
   - Large kernel (21) for global context
   - Physics-guided learning with auxiliary task


In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import time
from collections import deque
import math # PositionalEncoding에 필요

# ========================
# UCI-HAR 데이터 로드
# ========================
class UCIHARDataset(Dataset):
    def __init__(self, data_path, split='train'):
        base = Path(data_path) / split
        signals = []
        for sensor in ['body_acc', 'body_gyro', 'total_acc']:
            for axis in ['x', 'y', 'z']:
                file = base / 'Inertial Signals' / f'{sensor}_{axis}_{split}.txt'
                signals.append(np.loadtxt(file))

        self.X = np.stack(signals, axis=-1)
        self.y = np.loadtxt(base.parent / split / f'y_{split}.txt').astype(int) - 1

        try:
            self.subjects = np.loadtxt(base.parent / split / f'subject_{split}.txt').astype(int)
        except:
            self.subjects = np.ones(len(self.y))

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (torch.FloatTensor(self.X[idx]),
                torch.LongTensor([self.y[idx]])[0],
                self.subjects[idx])

# ========================
# 🔥 [신규] 트랜스포머 포지셔널 인코딩
# ========================
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0) # [1, max_len, d_model]
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x shape: [B, L, D]
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

# ========================
# 트랜스포머 Base 모델
# ========================
class BaseTransformerHAR(nn.Module):
    def __init__(self, input_dim=9, d_model=128, n_head=8, n_layers=4, n_classes=6, dropout=0.1):
        super().__init__()

        self.proj = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_head,
            dim_feedforward=d_model * 4,
            dropout=dropout,
            batch_first=True # (B, L, D) 입력을 위함
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=n_layers
        )

        self.norm_final = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, n_classes)

    def forward(self, x):
        # 이 기본 forward는 Physics 모델에서 오버라이드(override) 됩니다.
        x = self.proj(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.norm_final(x)
        return self.head(x[:, -1, :]) # 마지막 토큰의 출력을 분류에 사용

# ========================
# Physics-Guided Transformer HAR
# ========================
class PhysicsTransformerHAR(BaseTransformerHAR):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        d_model = self.head.in_features
        # 보조 태스크를 위한 '물리 헤드' 추가
        self.physics_head = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Linear(d_model // 2, 6) # 6 = acc (x,y,z) + gyro (x,y,z)
        )

    def forward(self, x, return_physics=False):
        # BaseTransformerHAR의 forward 로직 (Transformer 인코더 통과)
        x_feat = self.proj(x)
        x_feat = self.pos_encoder(x_feat)
        x_feat = self.transformer_encoder(x_feat)
        x_feat = self.norm_final(x_feat) # (B, L, D_model)

        # 1. 분류 헤드 (기본 태스크)
        logits = self.head(x_feat[:, -1, :]) # 마지막 타임스텝의 피처만 사용

        if return_physics:
            # 2. 물리 헤드 (보조 태스크)
            # (B, L, D_model) -> (B, L, 6)
            physics = self.physics_head(x_feat) # 모든 타임스텝의 피처 사용
            return logits, physics

        return logits

# ========================
# '물리 손실' 함수 (변경 없음)
# ========================
def physics_loss(physics_pred, X_raw):
    # physics_pred: [B, L, 6]
    # X_raw: [B, L, 9] (원본 입력)

    acc_pred = physics_pred[:, :, :3]
    gyro_pred = physics_pred[:, :, 3:6]

    acc_true = X_raw[:, :, 0:3]
    gyro_true = X_raw[:, :, 3:6]

    return F.smooth_l1_loss(acc_pred, acc_true) + F.smooth_l1_loss(gyro_pred, gyro_true)

# ========================
# '물리 기반' 학습 함수
# ========================
def train_physics(model, train_loader, test_loader, device, epochs=50):
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

    best_acc = 0

    print("="*50)
    print("5️⃣ Physics-Guided (Transformer) HAR 학습 시작")
    print("="*50)

    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        for X, y, _ in train_loader:
            X, y = X.to(device), y.to(device)

            optimizer.zero_grad()

            logits, physics = model(X, return_physics=True)

            loss_cls = F.cross_entropy(logits, y)
            loss_phys = physics_loss(physics, X)
            loss = loss_cls + 0.05 * loss_phys

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()
            correct += (logits.argmax(1) == y).sum().item()
            total += y.size(0)

        scheduler.step()

        model.eval()
        test_correct, test_total = 0, 0

        with torch.no_grad():
            for X, y, _ in test_loader:
                X, y = X.to(device), y.to(device)
                logits = model(X, return_physics=False)
                test_correct += (logits.argmax(1) == y).sum().item()
                test_total += y.size(0)

        train_acc = 100 * correct / total
        test_acc = 100 * test_correct / test_total
        best_acc = max(best_acc, test_acc)

        if (epoch + 1) % 10 == 0 or epoch == epochs - 1:
            print(f'[Physics] Epoch {epoch+1:02d}/{epochs}: Train Acc={train_acc:.2f}%, Test Acc={test_acc:.2f}% (Best: {best_acc:.2f}%)')

    return best_acc

# ========================
# 메인 실행 함수
# ========================
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # !!! 경로 수정 필요 !!!
    data_path = '/content/drive/MyDrive/Colab Notebooks/UCI-HAR/UCI-HAR'
    # data_path = './UCI-HAR' # 로컬 환경 예시

    try:
        train_ds = UCIHARDataset(data_path, split='train')
        test_ds = UCIHARDataset(data_path, split='test')
    except FileNotFoundError:
        print(f"Error: 데이터셋을 찾을 수 없습니다. 'data_path' 변수를 수정하세요.")
        print(f"현재 경로: {data_path}")
        return

    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

    print(f"Train: {len(train_ds)} samples, Test: {len(test_ds)} samples\n")

    # 5️⃣ Physics-Guided 모델 생성 및 학습 (🔥 Mamba -> Transformer로 변경)
    model_physics = PhysicsTransformerHAR(
        input_dim=9,    # UCI-HAR 9 채널
        d_model=128,
        n_head=8,       # 트랜스포머 헤드 수
        n_layers=4,
        n_classes=6     # UCI-HAR 6개 클래스
    ).to(device)

    acc_physics = train_physics(model_physics, train_loader, test_loader, device, epochs=50)

    # Summary
    print("\n" + "="*50)
    print("FINAL RESULT")
    print("="*50)
    print(f"5️⃣ Physics-Guided Transformer HAR (Best Test Acc): {acc_physics:.2f}%")

if __name__ == '__main__':
    main()

Using device: cuda
Train: 7352 samples, Test: 2947 samples

5️⃣ Physics-Guided (Transformer) HAR 학습 시작
[Physics] Epoch 10/50: Train Acc=94.78%, Test Acc=89.11% (Best: 90.43%)
[Physics] Epoch 20/50: Train Acc=95.23%, Test Acc=90.43% (Best: 91.41%)
[Physics] Epoch 30/50: Train Acc=96.08%, Test Acc=89.51% (Best: 91.41%)
[Physics] Epoch 40/50: Train Acc=96.98%, Test Acc=90.67% (Best: 91.55%)
[Physics] Epoch 50/50: Train Acc=97.58%, Test Acc=90.43% (Best: 91.55%)

FINAL RESULT
5️⃣ Physics-Guided Transformer HAR (Best Test Acc): 91.55%
